In [1]:
import pandas
import numpy

## Hybrid sample

In [2]:
filepath = 'D:/BG/Data/Processing/4_data.txt'
data = pandas.read_csv(filepath, sep='\t')
print(data.shape[0],'postings in full sample')
print(data.occupation.value_counts())
print(data.plant.nunique(),'plants in the full sample')
hybrid = data[data.plant_5==0]
print(hybrid.shape[0],'postings in hybrid sample')

3091485 postings in full sample
Manager       1229666
Operator      1043485
Engineer       610805
Technician     207529
Name: occupation, dtype: int64
337321 plants in the full sample
212822 postings in hybrid sample


## Table for hybrids by sub-period

In [23]:
cols = ['2014-2016-AM','2014-2016-TM','2017-2019-AM','2017-2019-TM']
df = hybrid.copy()
df['Year'] = pandas.to_datetime(df.JobDate).dt.year
df.loc[df.Year<2017,'Period'] = '2014-2016'
df.loc[df.Year>2016,'Period'] = '2017-2019'
N_plant = df.groupby(['Period'])[['plant']].nunique().T
N_plant = N_plant.applymap('{:.0f}'.format) 
N_plant = N_plant[['2014-2016','2014-2016','2017-2019','2017-2019']]
N_plant.columns = cols
N_firm = df.groupby(['Period'])[['firm']].nunique().T
N_firm = N_firm.applymap('{:.0f}'.format) 
N_firm = N_firm[['2014-2016','2014-2016','2017-2019','2017-2019']]
N_firm.columns = cols
N_jobs = df.groupby(['occupation','Period','TECH']).nunique()[['BGTJobId']].reindex(['Manager','Engineer','Technician','Operator'], level='occupation').unstack(level=[1,2])
N_jobs.columns = cols
N_jobs_perc = 100*N_jobs/N_jobs.sum()
N_jobs_perc.loc['Total',:] = N_jobs_perc.sum(axis=0)
N_jobs_perc = N_jobs_perc.applymap('{:.1f}'.format)
N_jobs_perc = N_jobs_perc.applymap(lambda x: x + '%')
N_jobs_perc['variable'] = 'Percentage'
N_jobs_perc.set_index('variable',append=True,inplace=True)
N_jobs.loc['Total',:] = N_jobs.sum(axis=0)
N_jobs = N_jobs.applymap('{:,.0f}'.format)
N_jobs['variable'] = 'Number'
N_jobs.set_index('variable',append=True,inplace=True)
table = pandas.concat([N_jobs,N_jobs_perc],axis=0).sort_index(kind='merge').reindex(['Manager','Engineer','Technician','Operator','Total'],axis=0,level='occupation')
N_firm['variable'] = 'firm'
N_plant['variable'] = 'plant'
N_firm.set_index(['variable'],append=True,inplace=True)
N_plant.set_index(['variable'],append=True,inplace=True)
table = pandas.concat([N_plant,N_firm,table],axis=0)
table

2014-2016-AM 2014-2016-TM 2017-2019-AM 2017-2019-TM
           variable                                                      
plant      plant               288          288          316          316
firm       firm                139          139          154          154
Manager    Number              192       41,085          789       47,735
           Percentage        17.6%        45.2%        22.3%        40.7%
Engineer   Number              713       32,623        1,965       45,352
           Percentage        65.2%        35.9%        55.4%        38.7%
Technician Number              104        4,521          353        6,270
           Percentage         9.5%         5.0%        10.0%         5.3%
Operator   Number               85       12,716          437       17,882
           Percentage         7.8%        14.0%        12.3%        15.3%
Total      Number            1,094       90,945        3,544      117,239
           Percentage       100.0%       100.0%       100.0%       100.0%

In [24]:
table.to_excel('Table2.xlsx')